# Q3.3 Estimate system parameters from least squared method

## Define functions

In [1]:
import numpy as np
from scipy.integrate import odeint

In [3]:
m1 = 1
m2 = 1
L1 = 0.5
L2 = 0.5
g = 9.81

def func(x,t,m1,m2,L1,L2,g):
    x1,x2,x3,x4 = x
    tau = np.array([[1], [1]])
    M = np.mat([[m1*(L1**2) + m2*(L1**2 + 2*L1*L2*np.cos(x2) + L2**2), m2*(L1*L2*np.cos(x2) + L2**2)], [m2*(L1*L2*np.cos(x2) + L2**2), m2*(L2**2)]])
    M_inv = np.mat([[1 / ((L1**2)*(m1 + m2*(np.sin(x2))**2)), -1*(L1*np.cos(x2) + L2) / ((L1**2)*L2*(m1 + m2*(np.sin(x2))**2))], [-1*(L1*np.cos(x2) + L2) / ((L1**2)*L2*(m1 + m2*(np.sin(x2))**2)), (m1*L1**2 + m2*(L1**2 + 2*L1*L2*np.cos(x2) + L2**2)) / (m2*(L1**2)*(L2**2)*(m1 + m2*(np.sin(x2))**2))]])
    C = np.array([[-1*m2*L1*L2*np.sin(x2)*(2*x3*x4 + x4**2)], [m2*L1*L2*(x3**2)*np.sin(x2)]])
    G = np.array([[(m1+m2)*L1*g*np.cos(x1) + m2*g*L2*np.cos(x1 + x2)], [m2*g*L2*np.cos(x1 + x2)]])
    x1_dot = x3
    x2_dot = x4
    x34_dot = np.dot(M.I, (tau - C - G))
    x3_dot = x34_dot[0,0]
    x4_dot = x34_dot[1,0]
    return [x1_dot, x2_dot, x3_dot, x4_dot]

t_range = 20 # Time length is 20s
dt = 0.001 # dt is 0.001s
times = np.arange(0,t_range,dt)

In [4]:
# g(θ) consists of theta1_dot2func and theta2_dot2func.
def theta1_dot2func(para, x1, x2, x3, x4, tau1, tau2):
    m1_pred, m2_pred, L1_pred, L2_pred = para
    M11_inv = 1 / ((L1_pred**2)*(m1_pred + m2_pred*(np.sin(x2))**2))
    M12_inv = -1*(L1_pred*np.cos(x2) + L2_pred) / ((L1_pred**2)*L2_pred*(m1_pred + m2_pred*(np.sin(x2))**2))
    C1 = -m2_pred*L1_pred*L2_pred*np.sin(x2)*(2*x3*x4 + x4**2)
    G1 = (m1_pred+m2_pred)*L1_pred*g*np.cos(x1) + m2_pred*g*L2_pred*np.cos(x1 + x2)
    C2 = m2_pred*L1_pred*L2_pred*(x3**2)*np.sin(x2)
    G2 = m2_pred*g*L2_pred*np.cos(x1 + x2)
    return M11_inv*(tau1 - C1 - G1) + M12_inv*(tau2 - C2 - G2)

def theta2_dot2func(para, x1, x2, x3, x4, tau1, tau2):
    m1_pred, m2_pred, L1_pred, L2_pred = para
    M21_inv = -1*(L1_pred*np.cos(x2) + L2_pred) / ((L1_pred**2)*L2_pred*(m1_pred + m2_pred*(np.sin(x2))**2))
    M22_inv = (m1_pred*L1_pred**2 + m2_pred*(L1_pred**2 + 2*L1_pred*L2_pred*np.cos(x2) + L2_pred**2)) / (m2_pred*(L1_pred**2)*(L2_pred**2)*(m1_pred + m2_pred*(np.sin(x2))**2))
    C1 = -m2_pred*L1_pred*L2_pred*np.sin(x2)*(2*x3*x4 + x4**2)
    G1 = (m1_pred+m2_pred)*L1_pred*g*np.cos(x1) + m2_pred*g*L2_pred*np.cos(x1 + x2)
    C2 = m2_pred*L1_pred*L2_pred*(x3**2)*np.sin(x2)
    G2 = m2_pred*g*L2_pred*np.cos(x1 + x2)
    return M21_inv*(tau1 - C1 - G1) + M22_inv*(tau2 - C2 - G2)

# error used by leastsq, while LScost used by minimize
def error(para, x1, x2, x3, x4, tau1, tau2, theta1_dot2, theta2_dot2):
    return np.sqrt((theta1_dot2 - theta1_dot2func(para, x1, x2, x3, x4, tau1, tau2))**2 + (theta2_dot2 - theta2_dot2func(para, x1, x2, x3, x4, tau1, tau2))**2)

## θ2_init = pi/2

In [5]:
#θ2_init = pi/2
input = np.array([0,np.pi/2])
x_init = np.hstack((input,np.array([0,0])))
x_traj = odeint(func, x_init, times, args=(m1,m2,L1,L2,g)) # Compute θ1 θ2 θ1_dot θ2_dot

In [6]:
import random
length = np.size(x_traj,0)
theta1_dot_dot = np.zeros(length) 
theta2_dot_dot = np.zeros(length)
for i in range(length):
    M_pred = np.mat([[m1*(L1**2) + m2*(L1**2 + 2*L1*L2*np.cos(x_traj[i,1]) + L2**2), m2*(L1*L2*np.cos(x_traj[i,1]) + L2**2)], [m2*(L1*L2*np.cos(x_traj[i,1]) + L2**2), m2*(L2**2)]])
    C_pred = np.mat([[-1*m2*L1*L2*np.sin(x_traj[i,1])*(2*x_traj[i,2]*x_traj[i,3] + x_traj[i,3]**2)], [m2*L1*L2*(x_traj[i,2]**2)*np.sin(x_traj[i,1])]])
    G_pred = np.mat([[(m1+m2)*L1*g*np.cos(x_traj[i,0]) + m2*g*L2*np.cos(x_traj[i,0] + x_traj[i,1])], [m2*g*L2*np.cos(x_traj[i,0] + x_traj[i,1])]])
    tau = np.mat([[1], [1]])
    theta12_dot2 = np.dot(M_pred.I, (tau - C_pred - G_pred)) # Compute θ1_dot_dot and θ2_dot_dot
    theta1_dot_dot[i] = theta12_dot2[0,0] # + random.uniform(-1,1) # Obtain mearsurement data of θ1_dot_dot by adding a noise
    theta2_dot_dot[i] = theta12_dot2[1,0] # + random.uniform(-1,1) # Obtain mearsurement data of θ2_dot_dot by adding a noise

In [11]:
from scipy.optimize import least_squares, leastsq

tau1 = np.ones(length) # τ1=τ2=0 when free falling
tau2 = np.ones(length)
para0 = [10, 10, 5, 10] # set initial values of parameters when theta2_init = pi/2
#para = least_squares(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))

In [12]:
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("theta2_init = ",input[1], " t_length = ",t_range)
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

theta2_init =  1.5707963267948966  t_length =  20
m1 =  0.9999997990181543  m2 =  0.9999998024549246  L1 =  0.500000004408748  L2 =  0.5000000114747059


### Try different initial values of m1, m2, L1, L2

In [16]:
#θ2_initial = pi/2
para0 = [1.5, 1.5, 1, 1] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  0.999999929174924  m2 =  0.9999999267874237  L1 =  0.4999999947695729  L2 =  0.4999999975697994


In [17]:
#θ2_initial = pi/2
para0 = [3, 400, 2, 14] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  1139.410455694629  m2 =  1111.142431374075  L1 =  0.4882703485458744  L2 =  0.5001606519247088


In [18]:
#θ2_initial = pi/2
para0 = [660, 6, 3, 6] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  0.9999998296260388  m2 =  0.9999998283090618  L1 =  0.5000000079582889  L2 =  0.500000013111332


## θ2_init = -pi/2

In [47]:
#θ2_init = -pi/2
input = np.array([0,-np.pi/2])
x_init = np.hstack((input,np.array([0,0])))
x_traj = odeint(func, x_init, times, args=(m1,m2,L1,L2,g)) # Compute θ1 θ2 θ1_dot θ2_dot

In [48]:
import random
length = np.size(x_traj,0)
theta1_dot_dot = np.zeros(length) 
theta2_dot_dot = np.zeros(length)
for i in range(length):
    M_pred = np.mat([[m1*(L1**2) + m2*(L1**2 + 2*L1*L2*np.cos(x_traj[i,1]) + L2**2), m2*(L1*L2*np.cos(x_traj[i,1]) + L2**2)], [m2*(L1*L2*np.cos(x_traj[i,1]) + L2**2), m2*(L2**2)]])
    C_pred = np.mat([[-1*m2*L1*L2*np.sin(x_traj[i,1])*(2*x_traj[i,2]*x_traj[i,3] + x_traj[i,3]**2)], [m2*L1*L2*(x_traj[i,2]**2)*np.sin(x_traj[i,1])]])
    G_pred = np.mat([[(m1+m2)*L1*g*np.cos(x_traj[i,0]) + m2*g*L2*np.cos(x_traj[i,0] + x_traj[i,1])], [m2*g*L2*np.cos(x_traj[i,0] + x_traj[i,1])]])
    tau = np.mat([[0], [0]])
    theta12_dot2 = np.dot(M_pred.I, (tau - C_pred - G_pred)) # Compute θ1_dot_dot and θ2_dot_dot
    theta1_dot_dot[i] = theta12_dot2[0,0] + random.uniform(-1,1) # Obtain mearsurement data of θ1_dot_dot by adding a noise
    theta2_dot_dot[i] = theta12_dot2[1,0] + random.uniform(-1,1) # Obtain mearsurement data of θ2_dot_dot by adding a noise

In [49]:
from scipy.optimize import leastsq, minimize

# attempt para0 = [9.5, 9.5, 2, 2]
tau1 = np.zeros(length) # τ1=τ2=0 when free falling
tau2 = np.zeros(length)
para0 = [9.5, 9.5, 2, 2] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
#para = least_squares(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  2.4655116757026905  m2 =  2.4655642163858538  L1 =  0.4998642732942489  L2 =  0.4998961214470978


In [50]:
from scipy.optimize import least_squares, leastsq, minimize

tau1 = np.zeros(length) # τ1=τ2=0 when free falling
tau2 = np.zeros(length)
para0 = [4.5, 4.5, 2.5, 2] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
#para = least_squares(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))

In [51]:
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("theta2_init = ",input[1], " t_length = ",t_range)
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

theta2_init =  -1.5707963267948966  t_length =  20
m1 =  1.2502348597552948  m2 =  1.2502621311734203  L1 =  0.49986482433116747  L2 =  0.4998965898251322


### Try different initial values of m1, m2, L1, L2

In [406]:
#θ2_initial = -pi/2
para0 = [1.5, 1.5, 1, 1] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  0.1620929115615569  m2 =  0.16207989887346652  L1 =  0.4999876216587387  L2 =  0.5000284521766708


In [409]:
#θ2_initial = -pi/2
para0 = [3, 400, 2, 14] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  2063.8466822421424  m2 =  2063.881803042769  L1 =  0.5000196691901436  L2 =  0.500082604380764


In [411]:
#θ2_initial = -pi/2
para0 = [60, 6, 3, 6] # set initial values of parameters
para = leastsq(error, para0, args=(x_traj[:length,0], x_traj[:length,1], x_traj[:length,2], x_traj[:length,3], tau1, tau2, theta1_dot_dot, theta2_dot_dot))
m1_pred,m2_pred,L1_pred,L2_pred = para[0]
print("m1 = ",m1_pred," m2 = ",m2_pred," L1 = ",L1_pred," L2 = ",L2_pred)

m1 =  8.932335200955812  m2 =  8.931629467083065  L1 =  0.4999877889082277  L2 =  0.5000288490810626
